**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**1) Clone, configure & compile Darknet**



In [ ]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

In [4]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
# Compile
!make

**2) Configure yolov3.cfg file**

In [6]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [7]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 70000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=35@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=35@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=35@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=120@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=120@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=120@' cfg/yolov3_training.cfg

**3) Create .names and .data files**

In [8]:
!echo -e 'Afgan Hound\nAustralian Terrier\nBorder Collie\nBouvier des Flandres\nClumber Spaniel\nCollie\nDalmatian\nDoberman\nEnglish Springer\nWire Fox Terrier\nFrench Bulldog\nGerman Shepherd\nGiant Schnauzer\nGolden Retiver\nIrish Setter\nKomondor\nKuvasz\nMaltese Dog\nOld English Sheepdog\nPapillon\nPomeranian\nPug\nRottweiler\nSaluki\nSchipperke\nScottish Deerhund\nSiberian Husky\nSoft Coated Wheaten Terrier\nStandard Poodle\nTibetan Terrier\nTibetan Mastiff\nVizsla\nWelsh Springer Spaniel\nWhippet\nYorkshire Terrier' > data/obj.names
!echo -e 'classes= 35\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/Dog_Detection_35_Classes/weights' > data/obj.data

**4) Save yolov3_training.cfg and obj.names files in Google drive**

In [ ]:
!cp cfg/yolov3_training.cfg /mydrive/Dog_Detection_35_Classes/yolov3_testing.cfg
!cp data/obj.names /mydrive/Dog_Detection_35_Classes/classes.txt

**5) Create a folder and unzip image dataset**

In [ ]:
!mkdir data/obj
!unzip /mydrive/Dog_Detection_35_Classes/images.zip -d data/obj

**6) Copy train.txt and test.txt files**

In [10]:
!cp /mydrive/Dog_Detection_35_Classes/train.txt data/train.txt
!cp /mydrive/Dog_Detection_35_Classes/test.txt data/test.txt

**7) Download pre-trained weights for the convolutional layers file**

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

**8) Start training**

In [ ]:
# START TRAINING
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

# CONTINUE TRAINING
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/Dog_Detection_35_Classes/weights/yolov3_training_last.weights -dont_show

**9) Test Model**

In [ ]:
!./darknet detector map data/obj.data cfg/yolov3_training.cfg /mydrive/Dog_Detection_35_Classes/weights/yolov3_training_last.weights